<a href="https://colab.research.google.com/github/githubforkj/signate/blob/main/mufj/02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# フォーラムで提示された強プレイヤーの写経

(完全理解も同時に行う)

## 準備

In [ ]:
# signateとの連携

!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 22.6 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=89806a12e8dceeecb72b384c246131a17f30717800efa826583ae33dea5f028a
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
requests 2.23.0 requires urllib3!

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 実装

In [ ]:
%%capture
!pip install transformers datasets sentencepiece

In [ ]:
!nvidia-smi

Sun Sep 11 22:45:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Import

In [ ]:
from pathlib import Path
from multiprocessing import cpu_count
import random
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score

from datasets import load_dataset, Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction
from transformers import set_seed

import torch
from torch import nn

### Config

In [ ]:
EXP_NAME = 'exp01'
INPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Input')
OUTPUT_DIR = Path('/content/drive/MyDrive/signate/mufj/Output')
DEBUG = True
TEXT_COLUMNS = ['goal','country','duration','category1','category2','html_content']
MODEL_NAME = 'microsoft/deberta-v3-base'
N_SPLIT = 5
TRN_FOLDS = [0,1,2,3,4]
MAX_LEN = 512
SEED = 3090


training_args = TrainingArguments(
    output_dir = str(OUTPUT_DIR/EXP_NAME),
    seed = SEED,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs = 4,
    weight_decay = 0.01,
    fp16 = True,
    gradient_accumulation_steps=1,
    evaluation_strategy='epoch',
    save_total_limit = 1,
    save_strategy = 'epoch',
    metric_for_best_model = 'f1_score',
    load_best_model_at_end = True,
    greater_is_better=True
)

### Cross Validation

In [ ]:
def create_folds(data, num_splits):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits = num_splits, shuffle=True, random_state=42)
    data_labels = data['state']

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f
    
    return data

data = pd.read_csv(INPUT_DIR / "train.csv")
data = create_folds(data, num_splits=N_SPLIT)
data.to_csv("train_folds.csv", index = False) #セッションでのみ作成される
print("Folds created sucessfully")

Folds created sucessfully


### Preprocess

In [ ]:
def text_to_input_ids(examples):
    return tokenizer(examples['text'] ,padding=False ,truncation=True, max_length=MAX_LEN)

def connect_text(df, text_cols, sep):
    text_df = df[text_cols].fillna('NAN').astype(str)
    connected_text = text_df[text_cols[0]].str.cat(text_df[text_cols[1:]], sep=sep)

    return connected_text

### Augumentation

expanding data

In [ ]:
class RandomMask:
    def __init__(self, tokenizer, prob=0.5, mask_prob=0.15):
        self.mask_token_id = tokenizer.mask_token_id
        self.prob = prob
        self.mask_prob = mask_prob
    
    def __call__(self, examples):
        if random.random() > self.prob:
            examples['input_ids'] = [self._mask(input_ids) for input_ids in examples['input_ids']]

        return examples
    
    def _mask(self, input_ids: list) -> list:
        length = len(input_ids)
        mask_idx = random.sample(range(1, length), int(length*self.mask_prob)) # random masking except [CLS]
        for i in mask_idx:
            input_ids[i] = self.mask_token_id

        return input_ids

### Trainer

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels').float()
        _inputs = {k: v for k, v in inputs.items() if k != 'labels'} #to not automatically calculate losses

        outputs = model(**_inputs)
        logits = outputs.get('logits')

        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(logits.view(-1), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true = y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(p: EvalPrediction):
    preds = sigmoid(p.predictions)
    labels = p.label_ids
    score = threshold_search(labels, preds)['f1']
    metrics = {'f1_score': score}

    return metrics
        

### Main

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv('train_folds.csv')
    test_df = pd.read_csv(INPUT_DIR / 'test.csv')
    # if DEBUG:
    #     df = df.head(50)
    #     test_df = test_df.head(50)
    #     training_args.num_train_epochs = 1

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #for dynamic padding
    train_transform = RandomMask(tokenizer, prob=0.5, mask_prob=0.1) #[MASK] augmentation

    df['text'] = connect_text(df, TEXT_COLUMNS, tokenizer.sep_token)
    df['label'] = df['state']
    ds = Dataset.from_pandas(df[['text','label','kfold']])
    ds = ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())

    test_df['text'] = connect_text(test_df, TEXT_COLUMNS, tokenizer.sep_token)
    test_ds = Dataset.from_pandas(test_df[['text']])
    test_ds = test_ds.map(text_to_input_ids, batched=True, num_proc=cpu_count())
    test_ds = test_ds.remove_columns(['text'])

    oof = []
    labels = []
    test_preds = []
    for fold in TRN_FOLDS:
        set_seed(SEED)
        ds_i = DatasetDict(
            {
                'train': ds.filter(lambda x: x['kfold']!=fold),
                'eval': ds.filter(lambda x: x['kfold']==fold),
            }
        )
        ds_i = ds_i.remove_columns(['text','kfold'])
        ds_i['train'].set_transform(train_transform)

        # train
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)
        training_args.output_dir = str((OUTPUT_DIR / EXP_NAME) / f'fold_{fold}')
        trainer = CustomTrainer(
            model = model,
            args = training_args,
            train_dataset = ds_i["train"],
            eval_dataset=ds_i["eval"],
            tokenizer = tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        trainer.train()

        # inference
        oof_i = trainer.predict(ds_i['eval'])
        test_preds_i = trainer.predict(test_ds).predictions
        oof.append(sigmoid(oof_i.predictions))
        labels.append(oof_i.label_ids)
        test_preds.append(test_preds_i)
    
    oof = np.vstack(oof).flatten()
    labels = np.hstack(labels)
    search_result = threshold_search(labels, oof)
    print('OOF Score: ', search_result['f1'], 'Threshold: ', search_result['threshold'])


    test_df['label'] = (np.mean(test_preds, axis=0) > search_result['threshold']).astype(int)
    test_df[['id','label']].to_csv((OUTPUT_DIR/EXP_NAME)/ 'sub.csv', header=False, index=False)

loading configuration file https://huggingface.co/microsoft/deberta-v3-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e6f9db57345f0f60c9f837fa97bcb27b1ed31e99feb33d732d7d8c80cb8f8459.de97182a9f32a68819030ba8f3f6ff2ba47276be3864425925523202f54cc79c
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

...

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/microsoft/deberta-v3-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/e6f9db57345f0f60c9f837fa97bcb27b1ed31e99feb33d732d7d8c80cb8f8459.de97182a9f32a68819030ba8f3f6ff2ba47276be3864425925523202f54cc79c
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input":

Epoch,Training Loss,Validation Loss,F1 Score
1,No log,0.468479,0.796688


***** Running Evaluation *****
  Num examples = 1958
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490
Configuration saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490/config.json
Model weights saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490/special_tokens_map.json
Deleting older checkpoint [/content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-3] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/signate/mufj/Output/exp01/fold_3/checkpoint-490 (score: 0.796688132474701).
***** Running Prediction *****